In [ ]:
import requests
from bs4 import BeautifulSoup
from enum import Enum
from datetime import datetime
import json
from tabulate import tabulate

red_rocks_url = 'https://www.redrocksonline.com/events/'
mission_ballroom_url = 'https://www.missionballroom.com/data/events-index.json'
tracks_file = 'tracks.json'

# vinyl, temple, church, ogden, fillmore, cervantes, summit, marquis
class Location(Enum):
    REDROCKS = 'Red Rocks'
    MISSION_BALLROOM = 'Mission Ballroom'

class Event:
    def __init__(self, date, headliner, openers, location):
        self.date = date
        self.headliner = headliner
        self.openers = openers
        self.location = location

def GetRedRocksEvents():
    events = []
    response = requests.get(red_rocks_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
    
        for event in soup.find_all('div', 'card-content'):
            date_text = event.contents[1].text.strip()
            if ' pm' not in date_text:
                continue #logic to skip duplicate entries - TODO: investigate and refactor
            date = datetime.strptime(date_text, "%a, %b %d, %I:%M %p")
            date = date.replace(year=datetime.now().year)
            headliner = event.contents[3].text.strip()
            openers = ''
            if len(event.contents) > 5:
                openers = event.contents[5].text.strip()
            events.append(Event(date, headliner, openers, Location.REDROCKS))
    else:
        print(f'Failed to retrieve page. Status code: {response.status_code}')

    return events

def GetMissionBallroomEvents():
    events = []
    response = requests.get(mission_ballroom_url)
    
    if response.status_code == 200:
        data = response.json()

        for event in data:
            date_text = f"{event['date']}, {event['doorDateTime']}"
            date = datetime.strptime(date_text, "%A, %B %d, %Y, %I:%M%p")
            headliner = event['title']
            openers = event['subtitle']
            events.append(Event(date, headliner, openers, Location.MISSION_BALLROOM))
    else:
        print(f'Failed to retrieve page. Status code: {response.status_code}')

    return events

def ReadTracksFromFile():
    with open(tracks_file, 'r') as file:
        return json.load(file)

def GetArtistSaveCounts(tracks):
    artists = {}

    for track in tracks:
        for artist in track['track']['artists']:
            name = artist['name']
            count = artists.get(name)
            if count is not None:
                artists[name] = count + 1
            else:
                artists[name] = 1

    # return sorted dictionary of artists in descending order of track save count
    return dict(sorted(artists.items(), key=lambda item: item[1], reverse=True))

def GetRelevantShows(shows, artists):
    relevant_shows = {}
    for key, value in artists.items():
        for show in shows:
            openers = [item.strip().lower() for item in show.openers.split(',')]
            key_lower = key.lower()
            if key_lower == show.headliner.strip().lower() or key_lower in openers:
                relevance = relevant_shows.get(show)
                if relevance is not None:
                    relevant_shows[show] = relevance + value
                else:
                    relevant_shows[show] = value

    # Return sorted dictionary of shows by descending relevance
    return dict(sorted(relevant_shows.items(), key=lambda item: item[1], reverse=True))

def PrintShows(show_dict):
    rows = [
        [relevance, show.headliner, show.openers, show.date.strftime("%m-%d-%Y %I:%M %p"), show.location.value]
        for show, relevance in show_dict.items()
    ]
    headers = ["Relevance", "Headliner", "Openers", "Date", "Location"]
    print(tabulate(rows, headers=headers, tablefmt="grid", maxcolwidths=50))

all_events = GetRedRocksEvents()
all_events += GetMissionBallroomEvents()
all_tracks = ReadTracksFromFile()
artist_counts = GetArtistSaveCounts(all_tracks)
relevant_shows = GetRelevantShows(all_events, artist_counts)
PrintShows(relevant_shows)